# Design and Sizing

## Imports

In [ ]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sympy as sp

from tabulate import tabulate

sys.path.append(os.path.abspath(".."))
from pyIC.utils import IcUtils as ic

## Gain

Gain is given by the transconductance of the input pair multiplied by the output resistance, $R_O$, given by the parallel $r_O$ of the cascode N and P "arms".

$$ A_V = g_{m1,2} R_O, \hspace{20pt} R_O = R_P || R_N $$

Equivalent output resistance of a cascode pair where $M_2$ is the "upper" device:

$$ R_{N,P} = (1 + g_{m2} r_{O2}) r_{O1} + r_{O2} $$ 

In [ ]:
gm2_p   = 1.6e-3
ro2_p   = 1 / 4.2e-6
ro1_p   = 1 / 7e-6

gm2_n   = 2.1e-3
ro2_n   = 1 / 7.1e-6
ro1_n   = 1 / 6e-6

RO_eq = (1 / (ro2_p * ro1_p * gm2_p) + 1 / (ro2_n * ro1_n * gm2_n))**(-1)

print(f'RO_eq: {ic.eng_format(RO_eq)}')

gm1     = 6e-3
gain    = ic.gain2db(RO_eq * gm1)
print(f'Gain: {gain}')

In [ ]:
gm2_p   = 1.6e-3
ro2_p   = 1 / 4.2e-6
ro1_p   = 1 / 7e-6
RO_p    = (1 + gm2_p * ro2_p) * ro1_p + ro2_p

gm2_n   = 2.1e-3
ro2_n   = 1 / 7.1e-6
ro1_n   = 1 / 6e-6
RO_n    = (1 + gm2_n * ro2_n) * ro1_n + ro2_n

print(f'RO_p: {ic.eng_format(RO_p)}')
print(f'RO_n: {ic.eng_format(RO_n)}')

RO_eq = (RO_p * RO_n) / (RO_p + RO_n)

print(f'RO_eq: {ic.eng_format(RO_eq)}')

gm1     = 6e-3
gain    = ic.gain2db(RO_eq * gm1)
print(f'Gain: {gain}')

In [ ]:
gm_p    = 2e-3
ro_p    = 1 / 6e-6
RO_p    = (1 + gm_p * ro_p) * ro_p + ro_p
ic.eng_format(RO_p)

In [ ]:
gm_n    = 2e-3
ro_n    = 1 / 7e-6
RO_n    = (1 + gm_n * ro_n) * ro_n + ro_n
ic.eng_format(RO_n)

In [ ]:
RO_eq = (RO_p * RO_n) / (RO_p + RO_n)
ic.eng_format(ic.gain2db(RO_eq * 6e-3))

In [ ]:
## --- Parameters ---
gmid1   = 13    # Input pair
gmid_p  = 3     # cascode pmos
gain_p  = 100
gmid_n  = 5     # cascode nmos

Av_db   = 100
Av      = ic.db2gain(Av_db)

# Load (buffer stage)
C_L     = 2 * 100e-15

# Required GBW (assuming single-pole)
GBW_min = 50e3 * Av

## --- Calculations ---
# Required gm1 for GBW
gm1_min = GBW_min * 2 * np.pi * C_L

# Required Ro
Ro_req  = Av / gm1_min

# Dominant pole
fp1     = 1 / (2 * np.pi * Ro_req * C_L)

# Required ID for min gm1
id1_min = gm1_min / gmid1

gm_p    = id1_min / gmid_p
ro_p    = gain_p / gm_p
RO_p    = (1 + gm_p * ro_p) * ro_p + ro_p

## --- Printing ---
table = [
    ['Ro_req'       , ic.eng_format(Ro_req)],
    ['Min GBW'      , ic.eng_format(GBW_min)],
    ['Dom pole'     , ic.eng_format(fp1)],
    ['Required gm1' , ic.eng_format(gm1_min)], 
    ['Min ID1'      , ic.eng_format(id1_min)],
    ['RO P-arm'     , ic.eng_format(RO_p)],
]

print(tabulate(table, tablefmt='rounded_outline', disable_numparse=True, colalign=['left', 'right']))

In [ ]:
## gm/id = 3, L = 1u
Cgg = sp.Symbol('Cgg')
eq = (5 * 0.5e-3) / (2 * np.pi * Cgg) - 3.73e9
sp.solve(eq)

### Input pair
- gmid = 13
- vgs  = 0.710 V
- ft   = 0.98  GHz
- W    = 441 u
- L    = 1 u

### Tail Source
- gmid = 5
- vgs  = 0.980
- gain = 57.2
- ft   = 2.59 G
- W    = 112
- L    = 1

### P-arm
- gmid = 3
- vgs  = 1.29
- gain = 100
- ft   = 0.95 G
- W    = 149
- L    = 1